In [10]:
from pacman import Individual, Population
from crossover import single_point_co, multi_point_co, uniform_co
from mutation import move_mutation, swap_mutation, inversion_mutation
from selection import fps, tournament_sel, boltzmann_selection, rank_based_selection

import pandas as pd
import numpy as np

# Dataframes Initialization (prepared for models with nr_gens=[100,200,300,500])

In [6]:
nr_gens = [100, 200, 300, 500]

dfs = {}

for nr in nr_gens:
    df_name = f"df_{nr}_gens"
    dfs[df_name] = pd.DataFrame({'Gens': range(1, nr + 1)}).set_index('Gens')

# Access the dataframes using their names
df_100_gens = dfs["df_100_gens"]
df_200_gens = dfs["df_200_gens"]
df_300_gens = dfs["df_300_gens"]
df_500_gens = dfs["df_500_gens"]



# Define the parameters for the grid search


In [7]:
import numpy as np
import pandas as pd

# Define the parameters for the grid search
nr_gens = [100, 300]
population_size = [200, 500]
population_moves = [500]
crossover_list = [single_point_co, multi_point_co, uniform_co]
crossover_prob = 0.9
mutation_list = [inversion_mutation]
mutation_prob = 0.1
selection_list = [fps, tournament_sel, boltzmann_selection, rank_based_selection]
elitism_selection = [True, False]
global_optimum = 23900

map = [['p', 'g', 'f', '*', 'g', '*', 'g', '*', 'g', '*', '*', 'g'], ['*', '*', '*', '*', '*', '*', '*', 'f', '*', '*', '*', '*'], ['*', 'g', '*', '*', 'g', '*', '*', 'g', '*', '*', 'g', 'f'], ['f', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', 'f'], ['*', '*', 'g', '*', 'g', 'f', '*', '*', 'g', '*', '*', 'g'], ['g', '*', '*', '*', '*', 'f', '*', '*', '*', '*', '*', '*'], ['f', '*', '*', '*', 'g', '*', 'g', '*', '*', 'g', '*', 'g'], ['g', '*', 'g', '*', 'f', '*', '*', '*', '*', '*', 'f', '*'], ['*', '*', 'f', '*', '*', 'g', 'f', 'g', '*', '*', '*', 'g'], ['g', '*', '*', 'g', '*', '*', '*', '*', '*', 'g', '*', '*'], ['*', '*', '*', '*', '*', 'f', 'f', '*', '*', '*', '*', '*'], ['*', 'g', '*', '*', 'g', '*', 'g', '*', '*', '*', 'g', '*']]

# Create a dictionary to store the results
results = {}


# Grid Searches (run only one for each time you initialize the Dataframes)

### Grid Search for Average Best Fitness, Standard Deviation and Cumulative Fitness Evaluations (prepared for running models with pop_size=[200,300,500] and equalize the computational cost of them)

In [ ]:
# Perform the grid search
for nr in nr_gens:
    df_name = f"df_{nr}_gens"
    df = dfs[df_name]  # Access the corresponding dataframe
    
    # Initialize a dictionary for the current dataframe's results
    df_results = {}
    
    for pop_size in population_size:
        for crossover_func in crossover_list:
            for mutation_func in mutation_list:
                for selection_func in selection_list:
                  for elitism_bol in elitism_selection:
                    for moves in population_moves:
                      # Initialize lists to store the results for each run and each generation
                      best_fitness_lists = [[] for _ in range(nr)]
                      fitness_evaluations_lists = [[] for _ in range(nr)]
                      
                      if pop_size == 200:
                          nr_runs = 75
                      elif pop_size == 300:
                          nr_runs = 50  
                      elif pop_size == 500:
                          nr_runs = 30
                      
                      for _ in range(nr_runs):  # Run specified number of times
                          # Create the initial population
                          pop = Population(size=pop_size, generated_map=map, moves=moves)
                          
                          # Evolve the population
                          pop.evolve(gens=nr, select=selection_func, mutate=mutation_func, crossover=crossover_func, 
                                     mut_prob=mutation_prob, xo_prob=crossover_prob, elitism=elitism_bol)
                          
                          fitness_evaluations = 0
                          # Get the best fitness and fitness evaluations for each generation
                          for i, individual in enumerate(pop.best_individuals_gen):
                              best_fitness = individual.fitness
                              fitness_evaluations += len(pop)
                              
                              # Append the results to the lists
                              best_fitness_lists[i].append(best_fitness)
                              fitness_evaluations_lists[i].append(fitness_evaluations)
                      
                      # Calculate the average of the best fitness and cumulative fitness evaluations for each generation
                      avg_best_fitness = [np.mean(best_fitness_lists[i]) for i in range(nr)]
                      cumulative_evaluations = [np.sum(fitness_evaluations_lists[i]) for i in range(nr)]
                      std_best_fitness = [np.std(best_fitness_lists[i]) for i in range(nr)]
                      
                      # Store the results in the dictionary
                      param_key = f"pop_size={pop_size}, nr_gens={nr}, "
                      df_results[param_key] = {"avg_best_fitness": avg_best_fitness, "std_best_fitness": std_best_fitness, "cumulative_evaluations": cumulative_evaluations}
                      print(param_key)
    
    # Store the results for the current dataframe in the overall results dictionary
    results[df_name] = df_results
    
    # Add the columns to the dataframe
    for param_key, result in df_results.items():
        df[param_key + "avg_best_fitness"] = result["avg_best_fitness"]
        df[param_key + "_std_best_fitness"] = result["std_best_fitness"]
        df[param_key + "cumulative_evaluations"] = result["cumulative_evaluations"]

### Grid Search for Average Best Fitness, Standard Deviation and Success Rates (In this case, all the models are run 100x)


In [ ]:
# Perform the grid search
for nr in nr_gens:
    df_name = f"df_{nr}_gens"
    df = dfs[df_name]  # Access the corresponding dataframe
    
    # Initialize a dictionary for the current dataframe's results
    df_results = {}
    
    for pop_size in population_size:
        for crossover_func in crossover_list:
            for mutation_func in mutation_list:
                for selection_func in selection_list:
                    for elitism_bol in elitism_selection:
                        for moves in population_moves:
                            # Initialize lists to store the results for each run and each generation
                            best_fitness_lists = [[] for _ in range(nr)]
                            fitness_evaluations_lists = [[] for _ in range(nr)]
                            success_rates = [0 for _ in range(nr)]
                            
                            nr_runs = 100
                            
                            for _ in range(nr_runs):  # Run specified number of times
                                print(pop_size, _)
                                # Create the initial population
                                pop = Population(size=pop_size, generated_map=map, moves=moves)
                                
                                # Evolve the population
                                pop.evolve(gens=nr, select=selection_func, mutate=mutation_func, crossover=crossover_func, 
                                           mut_prob=mutation_prob, xo_prob=crossover_prob, elitism=elitism_bol)
                                
                                # Initialize success counts for each generation
                                success_count = [0] * nr
                                
                                # Get the best fitness and fitness evaluations for each generation
                                for i, individual in enumerate(pop.best_individuals_gen):
                                    best_fitness = individual.fitness
                                    fitness_evaluations = len(pop)
                                    
                                    # Append the results to the lists
                                    best_fitness_lists[i].append(best_fitness)
                                    #fitness_evaluations_lists[i].append(fitness_evaluations)
                                    
                                    # Check if the individual reached the threshold fitness at each generation
                                    if best_fitness == global_optimum:
                                        success_rates[i] += 1
                                
                            
                            # Calculate the average of the best fitness and cumulative fitness evaluations for each generation
                            avg_best_fitness = [np.mean(best_fitness_lists[i]) for i in range(nr)]
                            #cumulative_evaluations = [np.sum(fitness_evaluations_lists[i]) for i in range(nr)]
                            std_best_fitness = [np.std(best_fitness_lists[i]) for i in range(nr)]
                            
                            # Store the results in the dictionary
                            param_key = f"pop_size={pop_size}, nr_gens={nr}, "
                            df_results[param_key] = {
                                "avg_best_fitness": avg_best_fitness,
                                "std_best_fitness": std_best_fitness,
                                #"cumulative_evaluations": cumulative_evaluations,
                                "success_rate": success_rates
                            }
    
    # Store the results for the current dataframe in the overall results dictionary
    results[df_name] = df_results
    
    # Add the columns to the dataframe
    for param_key, result in df_results.items():
        df[param_key + "avg_best_fitness"] = result["avg_best_fitness"]
        df[param_key + "_std_best_fitness"] = result["std_best_fitness"]
        #df[param_key + "cumulative_evaluations"] = result["cumulative_evaluations"]
        df[param_key + "success_rate"] = result["success_rate"]

# Export Excel with Results

In [7]:
with pd.ExcelWriter('results.xlsx') as writer:
    df_100_gens.to_excel(writer, sheet_name='df_100_gens')
    df_200_gens.to_excel(writer, sheet_name='df_200_gens')
    df_300_gens.to_excel(writer, sheet_name='df_300_gens')
    df_500_gens.to_excel(writer, sheet_name='df_500_gens')